# Recommender System Challenge

In [439]:
#Importing libraries
import pandas as pd
import scipy.sparse as sparse
import implicit
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [440]:
#Loading all the csv files
flickr_links = pd.read_csv("res2021/flickr_links.csv")
flickr_item_features = pd.read_csv("res2021/flickr_item_fea.csv")
flickr_train_data = pd.read_csv("res2021/flickr_train_data.csv")
flickr_test_data = pd.read_csv("res2021/flickr_test_data.csv")
flickr_user_features = pd.read_csv("res2021/flickr_user_fea.csv")
flickr_validation_data = pd.read_csv("res2021/flickr_validation_data.csv")
flickr_sample_solution_data = pd.read_csv("res2021/sample_solution_data.csv")

In [441]:
flickr_train_data.head()

,user_id,item_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [442]:
#unique user id's in the train data
flickr_train_data['user_id'].nunique()

3466

In [443]:
#unique item id's in the train data
flickr_train_data['item_id'].nunique()

9004

In [444]:
#maximum number in the user id's
max(flickr_train_data['user_id'])

3465

In [445]:
#minimum number in the user id's
min(flickr_train_data['user_id'])

0

In [446]:
# max and min item id value
max(flickr_train_data['item_id']),min(flickr_train_data['item_id'])

(9003, 0)

In [447]:
# finding the unique values for the rating
flickr_train_data['rating'].nunique()

1

In [448]:
# finding value counts for the ratings in train data
flickr_train_data['rating'].value_counts()

1    110129
Name: rating, dtype: int64

In [449]:
# creating sparse matrix
sparse_item_user = sparse.csr_matrix((flickr_train_data['rating'].astype(float), (flickr_train_data['item_id'], flickr_train_data['user_id'])))
sparse_user_item = sparse.csr_matrix((flickr_train_data['rating'].astype(float), (flickr_train_data['user_id'], flickr_train_data['item_id'])))

In [450]:
sparse_item_user

<9004x3466 sparse matrix of type '<class 'numpy.float64'>'
	with 110129 stored elements in Compressed Sparse Row format>

In [451]:
sparse_user_item

<3466x9004 sparse matrix of type '<class 'numpy.float64'>'
	with 110129 stored elements in Compressed Sparse Row format>

In [452]:
alpha = 30 #increasing the confidence in a preference with more interactions
data = (sparse_item_user * alpha).astype('double')

In [453]:
# Checking the datatype of data variable
type(data)

scipy.sparse.csr.csr_matrix

In [454]:
data[1,:].todense()

matrix([[30.,  0.,  0., ...,  0.,  0.,  0.]])

In [455]:
user_items = sparse_item_user.T.tocsr() #taking the transpose of sparse item user sparse matrix and turning into gensim document format.

In [456]:
# Intersection function
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return list(lst3)

# Collaborative Filtering for Implicit Feedback Datasets

## AlternatingLeastSquares

In [457]:
#fitting the data using ALS in Implicit package
model = implicit.als.AlternatingLeastSquares(factors=8, regularization=0.3, iterations=100)
model.fit(data)

  0%|          | 0/100 [00:00<?, ?it/s]

In [458]:
#unique user id's in the train data
flickr_train_data['user_id'].nunique()

3466

In [459]:
#unique item id's in the train data
flickr_train_data['item_id'].nunique()

9004

In [460]:
# Considering all the item recommendations for each user and finding the first top 15 recommended items 
# based on the item list in the test set for each user.
items = []
users = []
for i in range(3466):
    recommendations = model.recommend(i, user_items, N=9004)
    reco = []
    for recommendation in recommendations:
        reco.append(recommendation[0])
    score_list = intersection(reco,list(flickr_test_data[flickr_test_data['user_id']==i]['item_id']))
    items.append(score_list[:15])
    users.append([i,i,i,i,i,i,i,i,i,i,i,i,i,i,i])

In [461]:
# length of the items and users
len(items),len(users) #total 3466 lists in both users and items list

(3466, 3466)

In [462]:
# Flattening both the lists
flat_users = [item for sublist in users for item in sublist]
flat_items = [item for sublist in items for item in sublist]

In [463]:
#Zipping both the flattened lists
zipped_list = list(zip(flat_users,flat_items))

In [464]:
# Converting into csv file with the required format
final = pd.DataFrame(zipped_list, columns=['user_id','item_id'])
final.to_csv("final2.csv",index=False)

In [465]:
# 15 recommended items for the user '0'
model.recommend(0,user_items, N = 15)

[(4745, 0.32525814),
 (379, 0.31874484),
 (30, 0.31701577),
 (42, 0.30983996),
 (3610, 0.30440012),
 (3588, 0.30005553),
 (73, 0.29754508),
 (8416, 0.294688),
 (284, 0.28752795),
 (335, 0.28620598),
 (3612, 0.2860352),
 (56, 0.28253418),
 (3083, 0.282198),
 (4833, 0.28189278),
 (26, 0.28020525)]

In [466]:
# 15 recommended items after considering the test set items for user '0'
final[final['user_id']==0]

,user_id,item_id
0,0,7128
1,0,6343
2,0,1267
3,0,7091
4,0,3450
5,0,8275
6,0,838
7,0,3598
8,0,2607
9,0,1068


## LogisticMatrixFactorization

In [305]:
# fitting the data using LogisticMatrixFactorization in Implicit package
model1 = implicit.lmf.LogisticMatrixFactorization(factors = 5, regularization = 0.2, iterations = 50)
model1.fit(data)

100%|██████████| 50/50 [00:00<00:00, 61.33it/s]


In [306]:
#unique user id's in the train data
flickr_train_data['user_id'].nunique()

3466

In [307]:
#unique item id's in the train data
flickr_train_data['item_id'].nunique()

9004

In [308]:
# Considering all the item recommendations for each user and finding the first top 15 recommended items 
# based on the item list in the test set for each user.
items = []
users = []
for i in range(3466):
    recommendations = model1.recommend(i, user_items, N=9004)
    reco = []
    for recommendation in recommendations:
        reco.append(recommendation[0])
    score_list = intersection(reco,list(flickr_test_data[flickr_test_data['user_id']==i]['item_id']))
    items.append(score_list[:15])
    users.append([i,i,i,i,i,i,i,i,i,i,i,i,i,i,i])

In [309]:
len(items),len(users) #3466 lists in the items and users lists.

(3466, 3466)

In [310]:
# Flattening both the lists
flat_users = [item for sublist in users for item in sublist]
flat_items = [item for sublist in items for item in sublist]

In [311]:
#Zipping the flattened lists
zipped_list = list(zip(flat_users,flat_items))

In [312]:
# Converting into csv file with the required format
final = pd.DataFrame(zipped_list, columns=['user_id','item_id'])
final.to_csv("final1.csv",index=False)

In [313]:
# 15 recommended items for the user '0' based on LogisticMatrixFactorization
model1.recommend(0,user_items, N = 15)

[(1926, 11.578138),
 (237, 10.826538),
 (7611, 10.698066),
 (6219, 10.580546),
 (1502, 10.454693),
 (5292, 9.606646),
 (1082, 9.508778),
 (1100, 9.396305),
 (941, 9.358452),
 (5018, 9.245006),
 (2931, 9.237265),
 (4812, 9.231532),
 (5771, 9.215851),
 (2182, 9.142419),
 (777, 9.066832)]

In [314]:
# 15 recommended items after considering the test set items for user '0' using LogisticMatrixFactorization in implicit package
final[final['user_id']==0]

,user_id,item_id
0,0,7524
1,0,5029
2,0,691
3,0,3103
4,0,8552
5,0,838
6,0,8544
7,0,4713
8,0,1214
9,0,48


# Node Clustering in Graphs

In [232]:
#Importing libraries
import networkx as nx
import scipy as sp
import numpy as np
import torch

In [233]:
#Setting seed
torch.manual_seed(4179)

In [234]:
# Finding the nuumber of lines in the adjedges.txt file
number_of_lines = len(open('adjedges.txt').readlines())
print(number_of_lines)

18720


In [235]:
# Dictionary to store to nodes and their edges with other nodes
edges_dict = {}
with open('adjedges.txt', 'r') as adjedges:
    adjedges = [adjedges.readline().strip() for i in range(0,number_of_lines)]
    for each_link in adjedges:
        nodes = each_link.split(' ')[0]
        edges = each_link.split(' ')[1:]
        edges_dict[nodes] = edges

In [236]:
edges_dict

{'12828558': [],
 '66779408': [],
 '38902949': ['38998399'],
 '33450563': ['26547200'],
 '57470294': ['20968604'],
 '12016981': [],
 '59453341': [],
 '54791317': ['9564967', '14589786'],
 '38124547': [],
 '44569330': [],
 '4289706': ['48119811'],
 '34733903': [],
 '67315895': ['47074959',
  '36354718',
  '38323242',
  '40935482',
  '77107975',
  '38994139',
  '48871207',
  '75049959',
  '68339148',
  '62098573',
  '66337421',
  '78106693'],
 '47899676': [],
 '58164272': [],
 '76914035': [],
 '71555172': [],
 '6909690': ['56473434'],
 '47982561': ['26890304', '64372150'],
 '16864792': ['9684893', '47949816', '47887794'],
 '34534740': ['48119811', '16482894'],
 '27917678': ['44783549', '24245729', '33207680', '66903997'],
 '42441879': [],
 '20060161': [],
 '54709047': ['9633322', '32043684', '45159388', '54592233'],
 '17112646': ['17381885', '33338932'],
 '41499729': [],
 '39772251': [],
 '71431551': [],
 '34646799': [],
 '72286508': ['39750483'],
 '30054787': ['63059055',
  '62862974',


In [237]:
number_of_lines = len(open('labels.txt').readlines())
print(number_of_lines)

18720


In [238]:
# Dictionary to store to nodes and their labels
label_dict = {}
with open('labels.txt', 'r') as labels:
    labels = [labels.readline().strip() for i in range(0,number_of_lines)]
    for each_line in labels:
        node = each_line.split(' ')[0]
        label = each_line.split(' ')[1]
        label_dict[node] = int(label)

In [239]:
label_dict

{'12828558': 0,
 '66779408': 0,
 '38902949': 0,
 '33450563': 0,
 '57470294': 0,
 '12016981': 0,
 '59453341': 0,
 '54791317': 0,
 '38124547': 0,
 '44569330': 0,
 '4289706': 0,
 '34733903': 0,
 '67315895': 0,
 '47899676': 0,
 '58164272': 0,
 '76914035': 0,
 '71555172': 0,
 '6909690': 0,
 '47982561': 0,
 '16864792': 0,
 '34534740': 0,
 '27917678': 0,
 '42441879': 0,
 '20060161': 0,
 '54709047': 0,
 '17112646': 0,
 '41499729': 0,
 '39772251': 0,
 '71431551': 0,
 '34646799': 0,
 '72286508': 0,
 '30054787': 0,
 '14960319': 0,
 '20894906': 0,
 '43560951': 0,
 '12233872': 0,
 '58790184': 0,
 '41345168': 0,
 '53029990': 0,
 '17009966': 0,
 '23095245': 0,
 '39006078': 0,
 '22637451': 0,
 '1418126': 0,
 '61597421': 0,
 '62350848': 0,
 '38208385': 0,
 '54562160': 0,
 '47946998': 0,
 '71174685': 0,
 '40035502': 0,
 '14982527': 0,
 '18559780': 0,
 '72501271': 0,
 '10048225': 0,
 '40080547': 0,
 '44483771': 0,
 '6209405': 0,
 '32265946': 0,
 '6636178': 0,
 '38419714': 0,
 '39765555': 0,
 '47903875': 

In [240]:
# initialising undirected graph
g = nx.Graph()
ug = g.to_undirected()

In [241]:
# Creating a undirected graph with the nodes and edges
for node,edge in edges_dict.items():
    ug.add_node(node) 
    for edges in edge:
        ug.add_edge(node,edges)

In [242]:
# No of nodes and edges in undirected graph
print("No of nodes: ",len(ug.nodes()))
print("No of edges: ",len(ug.edges()))

No of nodes:  36928
No of edges:  54183


In [243]:
#Finding the no label nodes
no_label_nodes = []
for node in ug.nodes():
    if node in label_dict.keys():
        continue
    else:
        no_label_nodes.append(node)

In [244]:
#Removing no label nodes from the created undirected graph
for node in no_label_nodes:
    ug.remove_node(node)

In [246]:
# No of nodes and edges in undirected graph after removing no label nodes
print("No of nodes: ",len(ug.nodes()))
print("No of edges: ",len(ug.edges()))

No of nodes:  18720
No of edges:  9618


In [40]:
#Finding the true labels for the undirected graph
true_labels = []
for node in ug.nodes():
    if node in label_dict.keys():
        true_labels.append(label_dict[node])

In [41]:
# unique values in true_labels
np.unique(true_labels)

array([0, 1, 2, 3, 4])

In [42]:
# length of the true labels
len(true_labels)

18720

## Spectral Clustering for K clusters

#### Preprocessing and getting the laplacian matrix

In [44]:
#Laplacian matrix for 'ug' undirected graph
L = nx.laplacian_matrix(ug).astype(float)

#### Decomposition - Computing eigenvalues and eigenvectors of the matrix

In [45]:
w,v = sp.sparse.linalg.eigsh(L, k = 3,which='SM')

In [46]:
print(w)
print(v)

[-3.86204863e-13  3.14602632e-14  6.98483170e-04]
[[-2.86655492e-03  5.98309986e-03 -2.05553861e-15]
 [ 5.76174166e-06  1.30951803e-05 -6.31285603e-18]
 [ 7.09819813e-05 -1.39602761e-02  8.31134513e-15]
 ...
 [-1.02010836e-03  5.36606338e-03 -2.84139963e-15]
 [-1.66227638e-02  9.90473133e-03 -7.27869329e-15]
 [ 1.43742456e-03 -6.06371550e-03  1.47750498e-15]]


In [47]:
print(w.shape, v.shape)
X = v*w

(3,) (18720, 3)


In [60]:
X.shape

(18720, 3)

In [392]:
#K-means clusrtering
from sklearn.cluster import KMeans

kmeans = KMeans(init='k-means++', n_clusters=5, n_init=10)
kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_

In [394]:
np.unique(labels) # unique values in the labels which generated by k-means clustering

array([0, 1, 2, 3, 4], dtype=int32)

In [395]:
np.unique(true_labels) #unique values in the true labels

array([0, 1, 2, 3, 4])

In [397]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.18031566832673657

Received pretty low normalized mutual info score **0.18031566832673657** with the K-means clustering.

In [408]:
# MiniBatchKMeans clustering 
from sklearn.cluster import MiniBatchKMeans

kmeans1 = MiniBatchKMeans(init='k-means++', n_clusters=5,n_init=20, max_no_improvement=10, verbose=0)
kmeans1.fit_predict(X)
centroids = kmeans1.cluster_centers_
labels = kmeans1.labels_
error = kmeans1.inertia_

from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.17861568609642567

Received normalized mutual info score which is lower than the k-means clustering **0.17861568609642567** with MiniBatchKMeans clustering.

In [438]:
# KMedoids clustering with the nodes

# !pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids

kmedoids = KMedoids(n_clusters=5,init = "build")
kmedoids.fit_predict(X)
centroids = kmedoids.cluster_centers_
labels = kmedoids.labels_
error = kmedoids.inertia_

from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.1803156683267365

## Node Embedding Approach

In [56]:
# !pip install Node2Vec
from node2vec import Node2Vec

/Users/chaitanyamuvva/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [57]:
# pre-compute the probabilities and generate walks
node2vec = Node2Vec(ug, dimensions=64, walk_length=30, num_walks=200, workers=4)
# embedding the nodes
model = node2vec.fit(window=10,min_count=1,batch_words=4)

Computing transition probabilities:   0%|          | 0/18720 [00:00<?, ?it/s]

In [58]:
model.wv.get_vector('12828558') # generated vector of the node '12828558'

array([-0.00223427, -0.01299316, -0.00167405,  0.00092839, -0.00389336,
       -0.00281923, -0.00663311,  0.00712194, -0.01522242, -0.0039479 ,
        0.00550015,  0.01333038,  0.01327867,  0.01115854, -0.00229118,
       -0.00143999, -0.00222812,  0.00936443,  0.00690489, -0.0142027 ,
       -0.00204141, -0.01434673,  0.01322021,  0.00472553,  0.01137705,
       -0.00972108, -0.01245281,  0.01094597,  0.01463688, -0.01470541,
        0.00850832, -0.00659143,  0.00138195, -0.00497295, -0.0029316 ,
        0.00319988,  0.00607507, -0.00885461, -0.0123937 ,  0.01405009,
        0.00692496, -0.0115492 ,  0.00870955, -0.00422031, -0.01385356,
       -0.01535177, -0.00841296,  0.01353357,  0.00213606,  0.00220297,
       -0.01309112, -0.01181988, -0.00499053, -0.01079274,  0.0030236 ,
        0.00432758,  0.00878733, -0.00856931,  0.01466512,  0.00176107,
       -0.00371283,  0.00045177, -0.00917739,  0.00381868], dtype=float32)

In [59]:
model.wv.most_similar('12828558') # generated '12828558' similar nodes

[('54990868', 0.5018340945243835),
 ('70141902', 0.46260231733322144),
 ('11486624', 0.43991661071777344),
 ('21938655', 0.4212236702442169),
 ('50434168', 0.42107093334198),
 ('50245222', 0.40704208612442017),
 ('71171254', 0.4033096432685852),
 ('19704826', 0.4002232551574707),
 ('21003672', 0.39824992418289185),
 ('41223979', 0.39363744854927063)]

In [77]:
# Appending all the node vectors to the node_vector list
node_vectors = []
for node in ug.nodes():
    node_vectors.append(model.wv.get_vector(node))

In [78]:
len(node_vectors) #length of the node_vectors

18720

In [434]:
# K-means clustering with the node embeddings of each node
kmeans = KMeans(init='k-means++', n_clusters=5, n_init=10)
kmeans.fit_predict(node_vectors)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_

In [435]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.3088381573803444

There is an improvement in the normalized mutual info score when clustering using the node embeddings. The normalized mutual info score received from the above method is **0.3088381573803444**.

In [436]:
# MiniBatchKMeans clustering with the node embeddings of each node
from sklearn.cluster import MiniBatchKMeans

kmeans1 = MiniBatchKMeans(init='k-means++', n_clusters=5,n_init=20, max_no_improvement=10, verbose=0)
kmeans1.fit_predict(node_vectors)
centroids = kmeans1.cluster_centers_
labels = kmeans1.labels_
error = kmeans1.inertia_

from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.25899971266746497

K-means clustering is performing better compared to the MiniBatchKMeans when comparing with the normalized mutual info score. The normalized mutual info score for MiniBatchKmeans is **0.25899971266746497**

In [437]:
# KMedoids clustering with the node embeddings of each node

# !pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids

kmedoids = KMedoids(n_clusters=5,init = "build")
kmedoids.fit_predict(node_vectors)
centroids = kmedoids.cluster_centers_
labels = kmedoids.labels_
error = kmedoids.inertia_

from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(true_labels,labels)

0.29474648392478503

The normalized mutual info score for KMedoids is **0.29474648392478503**. The normalized mutual info score for the KMedoids is almost similar to the K-means clustering.